# LangChain GitHub Integration Demo

This demo was orignially presented during the Austin LangChain User Group meetup on August 7th 2024.

The demo shows a simple LangChain integration with a GitHub repository. The chain creates a GitHub pull request with generated code solutions based on a given GitHub issue inside a test project.

##Links:
presentation recording TODO

*   Presentation
*   [LangChain GitHub docs](https://python.langchain.com/v0.2/docs/integrations/tools/github/)
*   [LangSmith Trace](https://smith.langchain.com/public/ec13afae-28a6-4eb8-8cc4-36d4bd94a3d3/r)


##Setup


In [ ]:
%pip install --upgrade --quiet  pygithub langchain langchain-community langchain-openai pygithub python-dotenv

In [9]:
import os
from dotenv import load_dotenv
import getpass
from langchain.agents import AgentType
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain.agents import initialize_agent
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_core.tracers.context import tracing_v2_enabled

###Required Keys and environment variables

Either load the required variables from `.env` located in the project root by uncommenting `load_dotenv()` below or manually as we are doing here.

You will be required to setup app authentication with GitHub. Make sure to follow [these steps](https://python.langchain.com/v0.2/docs/integrations/tools/github/#2-create-a-github-app) to link a GitHub app to your repository.

Once you generate a private key for your app, you will need to load it into the notebook workspace and set `GITHUB_APP_PRIVATE_KEY` to the path of the .pem file.

In [ ]:
# load_dotenv()

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["GITHUB_APP_ID"] = "953998"
os.environ["GITHUB_APP_PRIVATE_KEY"] = "/content/codegen-langgraph.pem" #Path to your .pem key


os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "atx_lc_demo"

open_api_key = os.environ.get("OPEN_API_KEY")
langchain_api_key = os.environ.get("LANGCHAIN_API_KEY")
github_api_key = os.environ.get("GITHUB_API_KEY")

repo_issue_number = "1"
github_repo = "that1guy15/AIApp-Test"
github_bot_branch = f"CodeGenBot-Test-{repo_issue_number}"
github_base_branch = "main"


###Instantiate the LLM and GitHub Toolkit
`tools = toolkit.get_tools()` will load all available tools available to LangChain. This list of tools can be filtered and defined per agent if you want to limit what GitHub functionality each agent has available for use.  

In [21]:
llm = ChatOpenAI(model="gpt-4o")
github = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github)
tools = toolkit.get_tools()

print("Available tools:")
for tool in tools:
    print("\t" + tool.name)
print("\n\n")

Available tools:
	Get Issues
	Get Issue
	Comment on Issue
	List open pull requests (PRs)
	Get Pull Request
	Overview of files included in PR
	Create Pull Request
	List Pull Requests' Files
	Create File
	Read File
	Update File
	Delete File
	Overview of existing files in Main branch
	Overview of files in current working branch
	List branches in this repository
	Set active branch
	Create a new branch
	Get files from a directory
	Search issues and pull requests
	Search code
	Create review request





###Load and Prepare the GitHub issue

In [38]:
issue = github.get_issue(int(repo_issue_number))

To better control how the issue is presented to the agent we will use the following function to format the issue we just loaded.

In [32]:
def format_issue(issue):
    title = f"Title: {issue.get('title')}."
    opened_by = f"Opened by user: {issue.get('opened_by')}"
    body = f"Body: {issue.get('body')}"
    comments = issue.get("comments")  # often too long
    return "\n".join([title, opened_by, body, comments])

### Build the Agent Prompt

In [39]:
template = """
You are an experienced software engineer that creates Pull Requests based on GitHub issues.

Given the following GitHub issue:
{issue}

Your task is to:
1. Understand the issue and the changes required.
2. Write the necessary code to fix the issue including any existing tests.
If no test exist covering the issue, create a new test.
3. Create a new pull request on GitHub with the changes and link to the issue

Please provide a detailed explanation of the changes you made and why you made them in the pull request message.
"""
prompt = PromptTemplate(
    input_variables=["issue"],
    template=template,
)

final_prompt = prompt.format(issue=format_issue(issue))

print(final_prompt)


You are an experienced software engineer that creates Pull Requests based on GitHub issues.

Given the following GitHub issue:
Title: Filter option for get Users.
Opened by user: that1guy15
Body: Update the '/' GET endpoint to allow filtering based on all user properties. 

Here is the current function for reference:

```
@ns.route('/')
class UserList(Resource):
    '''Shows a list of all users, and lets you POST to add new users'''
    @ns.doc('list_users')
    @ns.marshal_list_with(user_model)
    def get(self):
        '''Fetch all users'''
        users = mongo.db.users.find()
        return list(users), 200
```
[]

Your task is to:
1. Understand the issue and the changes required.
2. Write the necessary code to fix the issue including any existing tests. 
If no test exist covering the issue, create a new test.
3. Create a new pull request on GitHub with the changes and link to the issue

Please provide a detailed explanation of the changes you made and why you made them in the pu

###Setup Shared memmory
We use a seperate LLM to summarize Agent output and interaction by utilizing `ConversationSummaryBufferMemory()`. This is important to help agents understand the full context of what has happened in the chain so far.

In [40]:
summarizer_llm = ChatOpenAI(model="gpt-4o", temperature=0)
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationSummaryBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    llm=summarizer_llm,
    max_token_limit=2_000,
)

###LangChain Agent Setup
Notice how `chat_history` and `final_prompt` are being passed into the agent under `agent_kwargs`

In [41]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,  # or pass a function that accepts the error and returns a string
    max_iterations=30,
    max_execution_time=None,
    early_stopping_method="generate",
    memory=memory,
    # trim_intermediate_steps=fancier_trim_intermediate_steps,
    agent_kwargs={
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"],
        "prefix": final_prompt,
    },
)

###Run the chain


In [ ]:
with tracing_v2_enabled(
    project_name=os.environ.get("LANGCHAIN_PROJECT"), tags=["PR_bot"]
) as cb:
    agent.run(final_prompt)

#Results

Once complete, you will now see a new pull request in your repository that is linked to the orignal issue. If the chain runs into issues completing the requests, it should respond in the comments of the issue.